# EDA - OpenAQ data

## Import library

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import requests
from pprint import pp
from tqdm.notebook import tqdm
import requests
import time
from zkyhaxpy import io_tools, pd_tools, gis_tools
from datetime import datetime, timedelta
import os

## Prepare GeoDataframe of OpenAQ data

In [2]:
if not os.path.exists('../data/gdf_openaq.gpkg'):
    print('Preparing gdf_openaq')
    df_openaq = pd.read_csv('../data/openaqi_thailand_pm25_data.csv', sep=',' )
    df_openaq['measurement_id'] = df_openaq.index

    df_openaq['geometry'] = 'POINT (' + df_openaq['long'].astype(str) + ' ' + df_openaq['lat'].astype(str) + ')'
    gdf_openaq = gis_tools.df_to_gdf(df_openaq, 'geometry')

    gdf_openaq['datetime'] = gdf_openaq['date_utc'].apply(lambda date_utc: pd.Timestamp(date_utc).tz_convert("UTC"))
    gdf_openaq['year'] = gdf_openaq['date_utc'].str.slice(0, 4).astype(int)
    gdf_openaq['month'] = gdf_openaq['date_utc'].str.slice(5, 7).astype(int)
    del(gdf_openaq['date_utc'])
    gdf_openaq = gdf_openaq.set_index('measurement_id')
    gdf_openaq.to_file('../data/gdf_openaq.gpkg')
    print('gdf_openaq has been saved.')
else:
    print('Loading gdf_openaq')
    gdf_openaq = gpd.read_file('../data/gdf_openaq.gpkg')
    print('gdf_openaq has been loaded.')
if gdf_openaq.index.name is None:
    gdf_openaq = gdf_openaq.set_index('measurement_id')

Loading gdf_openaq
gdf_openaq has been loaded.


In [3]:
gdf_openaq.head()

,locationId,location,parameter,value,unit,country,city,isMobile,isAnalysis,entity,sensorType,lat,long,datetime,year,month,geometry
measurement_id,,,,,,,,,,,,,,,,,
0,8893,Vientiane,pm25,10.0,µg/m³,TH,None,False,None,Governmental Organization,reference grade,17.896122,102.64,2019-06-30 00:00:00+00:00,2019,6,POINT (102.64000 17.89612)
1,8893,Vientiane,pm25,18.0,µg/m³,TH,None,False,None,Governmental Organization,reference grade,17.896122,102.64,2019-06-29 23:00:00+00:00,2019,6,POINT (102.64000 17.89612)
2,8893,Vientiane,pm25,17.0,µg/m³,TH,None,False,None,Governmental Organization,reference grade,17.896122,102.64,2019-06-29 22:00:00+00:00,2019,6,POINT (102.64000 17.89612)
3,8893,Vientiane,pm25,12.0,µg/m³,TH,None,False,None,Governmental Organization,reference grade,17.896122,102.64,2019-06-29 21:00:00+00:00,2019,6,POINT (102.64000 17.89612)
4,8893,Vientiane,pm25,15.0,µg/m³,TH,None,False,None,Governmental Organization,reference grade,17.896122,102.64,2019-06-29 20:00:00+00:00,2019,6,POINT (102.64000 17.89612)


In [4]:
gdf_province = gpd.read_file(r'..\data\thailandWithName.json')
gdf_chiangmai = gdf_province[gdf_province['name'] == 'Chiang Mai']
gdf_chiangmai

,name,geometry
14,Chiang Mai,"POLYGON ((99.39986 18.99843, 99.37143 18.97538..."


In [5]:
gdf_openaq.columns

Index(['locationId', 'location', 'parameter', 'value', 'unit', 'country',
       'city', 'isMobile', 'isAnalysis', 'entity', 'sensorType', 'lat', 'long',
       'datetime', 'year', 'month', 'geometry'],
      dtype='object')

In [9]:
gdf_openaq_first_data = gdf_openaq.groupby('locationId').agg(first_data=('datetime', 'min'))
gdf_openaq_first_data

,first_data
locationId,
135,2024-04-01 01:00:00+00:00
418,2021-06-23 22:00:00+00:00
701,2021-04-29 19:00:00+00:00
717,2021-04-29 19:00:00+00:00
906,2021-04-29 19:00:00+00:00
...,...
2692642,2024-04-01 15:00:00+00:00
2692643,2024-04-01 15:00:00+00:00
2692644,2024-04-01 15:00:00+00:00


In [6]:
gdf_openaq_location = gdf_openaq.drop(columns=['datetime', 'year', 'month', 'value', 'parameter', 'unit']).drop_duplicates()
gdf_openaq_location

# ['locationId', 'location', 'parameter', 'value', 'unit', 'country',
    #    'city', 'isMobile', 'isAnalysis', 'entity', 'sensorType', 'lat', 'long',
    #    'datetime', 'year', 'month', 'geometry']('')
ฟ

,locationId,location,country,city,isMobile,isAnalysis,entity,sensorType,lat,long,geometry
measurement_id,,,,,,,,,,,
0,8893,Vientiane,TH,None,False,None,Governmental Organization,reference grade,17.896122,102.640000,POINT (102.64000 17.89612)
14317,2407,Municipality Office Tungsadao,TH,None,False,None,Governmental Organization,reference grade,13.588500,101.286333,POINT (101.28633 13.58850)
14323,225634,"Rat Burana District Office, Bangkok",TH,None,False,None,Governmental Organization,reference grade,13.682136,100.506051,POINT (100.50605 13.68214)
14329,225593,Big C Supercenter Bang Na,TH,None,False,None,Governmental Organization,reference grade,13.667987,100.635313,POINT (100.63531 13.66799)
14335,2411,Health Promotion Hospital Banthasri,TH,None,False,None,Governmental Organization,reference grade,18.426783,99.757633,POINT (99.75763 18.42678)
...,...,...,...,...,...,...,...,...,...,...,...
217572,2692644,Kiak Kai Intersection,TH,None,False,None,Private Organization,low-cost sensor,13.796950,100.522272,POINT (100.52227 13.79695)
217726,2692642,Suriya Thep Alley,TH,None,False,None,Private Organization,low-cost sensor,13.749383,100.476910,POINT (100.47691 13.74938)
217880,2692643,Tang Trong Chit Alley,TH,None,False,None,Private Organization,low-cost sensor,13.768027,100.465332,POINT (100.46533 13.76803)


In [7]:
gdf_openaq_location_chiangmai = gdf_openaq_location[gdf_openaq_location.geometry.within(gdf_chiangmai.geometry.iloc[0])]
gdf_openaq_location_chiangmai

,locationId,location,country,city,isMobile,isAnalysis,entity,sensorType,lat,long,geometry
measurement_id,,,,,,,,,,,
14627,225669,"City Hall, Chiangmai",TH,None,False,None,Governmental Organization,reference grade,18.840732,98.969780,POINT (98.96978 18.84073)
14908,225579,Yupparaj Wittayalai School,TH,None,False,None,Governmental Organization,reference grade,18.790933,98.990000,POINT (98.99000 18.79093)
14954,225693,"Debaratana Vejjanukul Hospital, Mae Chaem",TH,None,False,None,Governmental Organization,reference grade,18.498333,98.379444,POINT (98.37944 18.49833)
15024,225583,Mobile Air Quality Monitoring unit No.04,TH,None,False,None,Governmental Organization,reference grade,18.804718,98.896851,POINT (98.89685 18.80472)
140358,387660,Mueang Na Subdistrict Municipality Office,TH,None,False,None,Governmental Organization,reference grade,19.593889,98.961667,POINT (98.96167 19.59389)
140822,523937,Tha Kham Municipality Office,TH,None,False,None,Governmental Organization,reference grade,18.202222,98.597222,POINT (98.59722 18.20222)
201417,1235999,NARIT ASTROPARK (Parking),TH,None,False,None,Governmental Organization,low-cost sensor,18.853455,98.957606,POINT (98.95761 18.85346)
202086,1236014,NARIT​-OPD,TH,None,False,None,Governmental Organization,low-cost sensor,18.852754,98.958347,POINT (98.95835 18.85275)
202254,1236024,สันโป่ง,TH,None,False,None,Governmental Organization,low-cost sensor,18.937511,98.956694,POINT (98.95669 18.93751)


In [12]:
gdf_openaq_location_chiangmai.merge(gdf_openaq_first_data, left_on='locationId', right_index=True)

,locationId,location,country,city,isMobile,isAnalysis,entity,sensorType,lat,long,geometry,first_data
measurement_id,,,,,,,,,,,,
14627,225669,"City Hall, Chiangmai",TH,None,False,None,Governmental Organization,reference grade,18.840732,98.969780,POINT (98.96978 18.84073),2021-04-29 19:00:00+00:00
14908,225579,Yupparaj Wittayalai School,TH,None,False,None,Governmental Organization,reference grade,18.790933,98.990000,POINT (98.99000 18.79093),2021-04-29 19:00:00+00:00
14954,225693,"Debaratana Vejjanukul Hospital, Mae Chaem",TH,None,False,None,Governmental Organization,reference grade,18.498333,98.379444,POINT (98.37944 18.49833),2021-04-29 19:00:00+00:00
15024,225583,Mobile Air Quality Monitoring unit No.04,TH,None,False,None,Governmental Organization,reference grade,18.804718,98.896851,POINT (98.89685 18.80472),2021-04-29 19:00:00+00:00
140358,387660,Mueang Na Subdistrict Municipality Office,TH,None,False,None,Governmental Organization,reference grade,19.593889,98.961667,POINT (98.96167 19.59389),2023-02-01 01:00:00+00:00
140822,523937,Tha Kham Municipality Office,TH,None,False,None,Governmental Organization,reference grade,18.202222,98.597222,POINT (98.59722 18.20222),2023-02-01 01:00:00+00:00
201417,1235999,NARIT ASTROPARK (Parking),TH,None,False,None,Governmental Organization,low-cost sensor,18.853455,98.957606,POINT (98.95761 18.85346),2024-04-01 01:00:00+00:00
202086,1236014,NARIT​-OPD,TH,None,False,None,Governmental Organization,low-cost sensor,18.852754,98.958347,POINT (98.95835 18.85275),2024-04-01 01:00:00+00:00
202254,1236024,สันโป่ง,TH,None,False,None,Governmental Organization,low-cost sensor,18.937511,98.956694,POINT (98.95669 18.93751),2024-04-01 01:00:00+00:00


In [14]:
gdf_openaq_location_chiangmai.to_file('gdf_openaq_location_chiangmai.gpkg')